In [1]:
import sys
sys.path.append('..')
from model.models_dgl import FixedNet2
from build_graph import build_graph
import dgl
import torch
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F

Using backend: pytorch


In [9]:
GRAPHS_NUM = 10000
LABEL = 3
NODES_NUM = 25
device = torch.device('cuda')
fixed_model = FixedNet2(1,4,2,False,'GraphConvWL')
fixed_model.set_paramerters()
fixed_model.to(device)

FixedNet2(
  (convs): ModuleList(
    (0): GraphConvWL(
      (conv_from_neigh): GraphConv(in=1, out=1, normalization=none, activation=None)
      (conv_from_self): Linear(in_features=1, out_features=1, bias=False)
    )
    (1): GraphConvWL(
      (conv_from_neigh): GraphConv(in=1, out=1, normalization=none, activation=None)
      (conv_from_self): Linear(in_features=1, out_features=1, bias=False)
    )
  )
  (fc1): Linear(in_features=1, out_features=8, bias=True)
  (output): Linear(in_features=8, out_features=4, bias=True)
)

In [21]:
def generate_single_sample(label, perturb_type, nodes_num = 25, m = 1, perturb_dic = {}, 
seed = None, no_attach_init_nodes=False):
    '''
    return a networkx instance
    '''
    basis_type = "ba"
    which_type = label
    if which_type == 0:
        if perturb_type == 0:
            G, role_id, plug_id = build_graph(nodes_num, basis_type, [], start = 0, m = m, seed = seed, no_attach_init_nodes=no_attach_init_nodes)
        else:
            G, role_id, plug_id = build_graph(nodes_num - perturb_type, basis_type, [[perturb_dic[perturb_type]]], start = 0, m = m, seed = seed, no_attach_init_nodes=no_attach_init_nodes)
    else:
        list_shapes = [["house"]] * (which_type - 1) + [["five_cycle"]] * (3 - which_type)
        if perturb_type != 0:
            list_shapes = list_shapes + [[perturb_dic[perturb_type]]]
        G, role_id, plug_id = build_graph(nodes_num-10-perturb_type, basis_type, list_shapes, start = 0, m = m, seed = seed, no_attach_init_nodes=no_attach_init_nodes)
    return G
result = []
for _ in range(GRAPHS_NUM):
    G = generate_single_sample(LABEL, 0, nodes_num = NODES_NUM, m = 6, perturb_dic = {4:'square_diagonal'}, no_attach_init_nodes = True)
    g = dgl.from_networkx(G)
    g = g.to(device)
    x = torch.ones((25,1)).to(device)
    if LABEL!=0:
        motif_mask = [False for _ in range(g.num_edges())]
        correct_ids = list(range(NODES_NUM-10, NODES_NUM))
        for i in range(g.num_edges()):
            u = g.edges()[0][i].item()
            v = g.edges()[1][i].item()
            if u in correct_ids or v in correct_ids:
                motif_mask[i] = True
    input_mask = torch.tensor(np.random.choice(np.arange(0,1,100), size = g.num_edges(), replace=True), dtype = torch.float32).to(device)
    input_mask[motif_mask] = 1.0
    result.append(np.argmax(fixed_model(g,x,input_mask).cpu().detach().numpy()))

In [3]:
from collections import Counter
Counter(result)

NameError: name 'result' is not defined

In [10]:
from build_graph import BA4labelDataset
fixed_model.set_random_gnn_parameters()
k = 0
for p in fixed_model.parameters():
    if k >= 6:
        p.requires_grad = False
    k += 1


In [11]:

BATCH_SIZE = 16
EPOCHS = 100
fixed_model.train()
optimizer = torch.optim.Adam(fixed_model.parameters(), lr=0.0003, weight_decay=0)
data = BA4labelDataset(graphs_num = GRAPHS_NUM, m = 6,nodes_num=NODES_NUM, perturb_dic = {}, no_attach_init_nodes = True)
dataloader = dgl.dataloading.GraphDataLoader(data, batch_size = BATCH_SIZE, shuffle = True)
data2 = BA4labelDataset(graphs_num = GRAPHS_NUM, m = 6,nodes_num=NODES_NUM, perturb_dic = {}, no_attach_init_nodes = True)
testdataloader = dgl.dataloading.GraphDataLoader(data2, batch_size = BATCH_SIZE, shuffle = True)

pbar = tqdm(range(EPOCHS))
for _ in pbar:
    loss_all = 0
    for g, labels in dataloader:
        g = g.to(device)
        optimizer.zero_grad()
        output = fixed_model(g, g.ndata['x'])
        loss = F.nll_loss(output, labels.to(device))
        loss.backward()
        loss_all += loss.item()
        optimizer.step()
    train_loss = loss_all/len(dataloader)
    #train_acc
    fixed_model.eval()
    correct = 0
    total = 0
    for g, labels in dataloader:
        g = g.to(device)
        output = fixed_model(g, g.ndata['x'])
        pred = output.max(dim=1)[1]
        eq_pred = pred.eq(labels.to(device))
        correct += eq_pred.sum().item()
        total += len(labels.to(device))
    train_acc = correct/total

    #test_acc
    fixed_model.eval()
    correct = 0
    total = 0
    for g, labels in testdataloader:
        g = g.to(device)
        output = fixed_model(g, g.ndata['x'])
        pred = output.max(dim=1)[1]
        eq_pred = pred.eq(labels.to(device))
        correct += eq_pred.sum().item()
        total += len(labels.to(device))
    test_acc = correct/total

    pbar.set_postfix(train_loss=train_loss, train_acc = train_acc, test_acc = test_acc)

  3%|▎         | 3/100 [00:19<10:18,  6.38s/it, test_acc=1, train_acc=1, train_loss=-1]


KeyboardInterrupt: 